In [3]:
import os
from os.path import join
import cv2
import numpy as np

path = '/home/zhong/Dataset/SynBlur'
dirs = os.listdir(path)
for dir in dirs:
    dir_path = join(path, dir)
    img_files = os.listdir(dir_path)
    for img_file in img_files:
        if not img_file.endswith('raw'):
            continue
        img_path = join(dir_path, img_file)
        img = np.fromfile(img_path, dtype='uint8')
        img = np.stack((img[0::3].reshape((512,512)), img[1::3].reshape((512,512)), img[2::3].reshape(512,512)), axis=-1)
        img = img[:,:,::-1]
        save_file = '{:08d}.png'.format(int(img_file.split('.')[0][-5:]))
        save_path = join(dir_path, save_file)
        cv2.imwrite(save_path, img)
        # cv2.imshow('foo', img[:,:,:])
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
    print('{} finish'.format(dir))

C001H002S0006 finish
C001H002S0015 finish
C001H002S0010 finish
C001H002S0024 finish
C001H002S0027 finish
C001H002S0018 finish
C001H002S0009 finish
C001H002S0002 finish
C001H002S0023 finish
C001H002S0005 finish
C001H002S0028 finish
C001H002S0014 finish
C001H002S0026 finish
C001H002S0021 finish
C001H002S0017 finish
C001H002S0003 finish
C001H002S0025 finish
C001H002S0007 finish
C001H002S0012 finish
C001H002S0013 finish
C001H002S0019 finish
C001H002S0022 finish
C001H002S0011 finish
C001H002S0016 finish
C001H002S0020 finish
C001H002S0004 finish
C001H002S0001 finish
C001H002S0008 finish


In [22]:
import os
from os.path import join

def syn_blur(img_paths: list) -> (np.array, np.array):
    num_imgs = len(img_paths)
    img_blur = cv2.imread(img_paths[0]).astype(np.float)
    for img_path in img_paths[1:]:
        img_blur += cv2.imread(img_path).astype(np.float)
    img_blur /= num_imgs
    img_blur = img_blur.astype(np.uint8)
    img_sharp = cv2.imread(img_paths[num_imgs//2])
    assert img_blur.dtype == img_sharp.dtype

    return img_blur, img_sharp

def adjust_gamma(image, gamma=1.0):
    inv_gamma = 1.0/gamma
    table = []
    for i in range(256):
        table.append(((i / 255.0) ** inv_gamma) * 255)
    table = np.array(table).astype("uint8")
    return cv2.LUT(image, table)


path = '/home/zhong/Dataset/SynBlur'
dirs = os.listdir(path)
dirs = sorted(dirs)
ds_path = '/home/zhong/Dataset/syn_blur_30/'
syn_frames = 65
for dir in dirs:
    dir_path = join(path, dir)
    save_dir_path = join(ds_path, dir)
    os.makedirs(save_dir_path, exist_ok=True)
    os.makedirs(join(save_dir_path, 'Blur'), exist_ok=True)
    os.makedirs(join(save_dir_path, 'Sharp'), exist_ok=True)
    img_files = os.listdir(dir_path)
    img_files = [img_file for img_file in img_files if img_file.endswith('png')]
    img_files = sorted(img_files, key=lambda x: int(x.split('.')[0]))
    img_files = [join(dir_path, img_file) for img_file in img_files]
    for i in range(len(img_files)//syn_frames):
        img_paths = img_files[i*syn_frames:(i+1)*syn_frames]
        img_blur, img_sharp = syn_blur(img_paths)
        img_blur, img_sharp = adjust_gamma(img_blur, gamma=2.2), adjust_gamma(img_sharp, gamma=2.2)
        # cv2.imshow('blur', img_blur)
        # cv2.imshow('sharp', img_sharp)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
        # break
        cv2.imwrite(join(save_dir_path, 'Blur', '{:08d}.png'.format(i)), img_blur)
        cv2.imwrite(join(save_dir_path, 'Sharp', '{:08d}.png'.format(i)), img_sharp)